In [29]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [30]:
absolute_path = '/Users/jonathancheng/Downloads/backtesting_data.xlsx'
excel_data = pd.read_excel(absolute_path, sheet_name='Sheet1')
excel_data = pd.DataFrame(excel_data)
excel_data['Date'] = pd.to_datetime(excel_data['Date'])

excel_data.set_index('Date', inplace=True)
full_index = pd.date_range(start=excel_data.index.min(), end=excel_data.index.max(), freq='D')
excel_data = excel_data.reindex(full_index)
excel_data = excel_data.ffill()
excel_data.reset_index(inplace=True)
excel_data.rename(columns={'index': 'Date'}, inplace=True)
excel_data = excel_data[['Date', 'SP500', 'DXY', 'US10Y']]

excel_data

,Date,SP500,DXY,US10Y
0,1927-12-30,17.660000,NaN,NaN
1,1927-12-31,17.660000,NaN,NaN
2,1928-01-01,17.660000,NaN,NaN
3,1928-01-02,17.660000,NaN,NaN
4,1928-01-03,17.760000,NaN,NaN
...,...,...,...,...
35196,2024-05-10,5222.680176,105.309998,4.504
35197,2024-05-11,5222.680176,105.309998,4.504
35198,2024-05-12,5222.680176,105.309998,4.504
35199,2024-05-13,5221.419922,105.220001,4.481


Define US president eletion dates

In [31]:
election_dates = [
    '1928-11-06', '1932-11-08', '1936-11-03', '1940-11-05', '1944-11-07', 
    '1948-11-02', '1952-11-04', '1956-11-06', '1960-11-08', '1964-11-03', 
    '1968-11-05', '1972-11-07', '1976-11-02', '1980-11-04', '1984-11-06', 
    '1988-11-08', '1992-11-03', '1996-11-05', '2000-11-07', '2004-11-02', 
    '2008-11-04', '2012-11-06', '2016-11-08', '2020-11-03'
]

democratic_dates = [
    '1932-11-08', '1936-11-03', '1940-11-05', '1944-11-07', '1948-11-02', 
    '1960-11-08', '1964-11-03', '1976-11-02', '1992-11-03', '1996-11-05', 
    '2008-11-04', '2012-11-06', '2020-11-03'
]

republican_dates = [
    '1928-11-06', '1952-11-04', '1956-11-06', '1968-11-05', '1972-11-07', 
    '1980-11-04', '1984-11-06', '1988-11-08', '2000-11-07', '2004-11-02', 
    '2016-11-08'
]

election_dates = pd.to_datetime(election_dates)
democratic_dates = pd.to_datetime(democratic_dates)
republican_dates = pd.to_datetime(republican_dates)

All eletion

In [32]:
all_election_df = excel_data[excel_data['Date'].isin(election_dates)].copy()
all_election_df['SP500選舉年初'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'SP500'].values[0]
)

all_election_df['SP500選舉年尾'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'SP500'].values[0]
)

def get_SP500_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'SP500'].values[0]

all_election_df['SP500選後半年'] = all_election_df['Date'].apply(lambda x: get_SP500_later(x, excel_data, 6))
all_election_df['SP500選後一年'] = all_election_df['Date'].apply(lambda x: get_SP500_later(x, excel_data, 12))
all_election_df['SP500選後兩年'] = all_election_df['Date'].apply(lambda x: get_SP500_later(x, excel_data, 24))

all_election_df['DXY選舉年初'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'DXY'].values[0]
)

all_election_df['DXY選舉年尾'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'DXY'].values[0]
)

def get_DXY_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'DXY'].values[0]

all_election_df['DXY選後半年'] = all_election_df['Date'].apply(lambda x: get_DXY_later(x, excel_data, 6))
all_election_df['DXY選後一年'] = all_election_df['Date'].apply(lambda x: get_DXY_later(x, excel_data, 12))
all_election_df['DXY選後兩年'] = all_election_df['Date'].apply(lambda x: get_DXY_later(x, excel_data, 24))

all_election_df['US10Y選舉年初'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'US10Y'].values[0]
)
all_election_df['US10Y選舉年尾'] = all_election_df['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'US10Y'].values[0]
)


def get_US10Y_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'US10Y'].values[0]

all_election_df['US10Y選後半年'] = all_election_df['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 6))
all_election_df['US10Y選後一年'] = all_election_df['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 12))
all_election_df['US10Y選後兩年'] = all_election_df['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 24))


In [33]:
all_election_df['SP500_選舉年變化(%)'] = ((all_election_df['SP500選舉年尾'] - all_election_df['SP500選舉年初']) / all_election_df['SP500選舉年初'] * 100).round(0).astype(int).astype(str) + '%'
all_election_df['SP500_選後半年變化(%)'] = ((all_election_df['SP500選後半年'] - all_election_df['SP500']) / all_election_df['SP500'] * 100).round(0).astype(int).astype(str) + '%'
all_election_df['SP500_選後一年變化(%)'] = ((all_election_df['SP500選後一年'] - all_election_df['SP500']) / all_election_df['SP500'] * 100).round(0).astype(int).astype(str) + '%'
all_election_df['SP500_選後兩年變化(%)'] = ((all_election_df['SP500選後兩年'] - all_election_df['SP500']) / all_election_df['SP500'] * 100).round(0).astype(int).astype(str) + '%'

all_election_df['DXY_選舉年變化'] = ((all_election_df['DXY選舉年尾'] - all_election_df['DXY選舉年初']))
all_election_df['DXY_選後半年變化'] = ((all_election_df['DXY選後半年'] - all_election_df['DXY']))
all_election_df['DXY_選後一年變化'] = ((all_election_df['DXY選後一年'] - all_election_df['DXY']))
all_election_df['DXY_選後兩年變化'] = ((all_election_df['DXY選後兩年'] - all_election_df['DXY']))

all_election_df['US10Y_選舉年變化'] = (all_election_df['US10Y選舉年尾'] - all_election_df['US10Y選舉年初'])
all_election_df['US10Y_選後半年變化'] = (all_election_df['US10Y選後半年'] - all_election_df['US10Y'])
all_election_df['US10Y_選後一年變化'] = (all_election_df['US10Y選後一年'] - all_election_df['US10Y'])
all_election_df['US10Y_選後兩年變化'] = (all_election_df['US10Y選後兩年'] - all_election_df['US10Y'])

all_election_df_change = all_election_df[['Date', 'SP500','DXY', 'US10Y', 'SP500_選舉年變化(%)', 'SP500_選後半年變化(%)', 'SP500_選後一年變化(%)', 'SP500_選後兩年變化(%)','DXY_選舉年變化', 'DXY_選後半年變化', 'DXY_選後一年變化', 'DXY_選後兩年變化','US10Y_選舉年變化', 'US10Y_選後半年變化', 'US10Y_選後一年變化', 'US10Y_選後兩年變化']]
all_election_df_change

,Date,SP500,DXY,US10Y,SP500_選舉年變化(%),SP500_選後半年變化(%),SP500_選後一年變化(%),SP500_選後兩年變化(%),DXY_選舉年變化,DXY_選後半年變化,DXY_選後一年變化,DXY_選後兩年變化,US10Y_選舉年變化,US10Y_選後半年變化,US10Y_選後一年變化,US10Y_選後兩年變化
312,1928-11-06,22.240000,NaN,NaN,38%,18%,-7%,-25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1775,1932-11-08,7.240000,NaN,NaN,-15%,15%,35%,26%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3231,1936-11-03,17.170000,NaN,NaN,28%,-4%,-33%,-23%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4694,1940-11-05,11.160000,NaN,NaN,-15%,-16%,-13%,-15%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6157,1944-11-07,12.960000,NaN,NaN,14%,15%,32%,14%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7613,1948-11-02,16.700001,NaN,NaN,-1%,-11%,-2%,18%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9076,1952-11-04,24.600000,NaN,NaN,12%,2%,0%,33%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10539,1956-11-06,47.599998,NaN,NaN,3%,-3%,-15%,10%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12002,1960-11-08,55.110001,NaN,NaN,-3%,21%,29%,6%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13458,1964-11-03,85.180000,NaN,4.150,13%,5%,8%,-5%,NaN,NaN,NaN,NaN,0.070,0.060000,0.280,0.910000


Democratic win

In [34]:
democratic = excel_data[excel_data['Date'].isin(democratic_dates)].copy()
democratic['SP500選舉年初'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'SP500'].values[0]
)

democratic['SP500選舉年尾'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'SP500'].values[0]
)

def get_SP500_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'SP500'].values[0]

democratic['SP500選後半年'] = democratic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 6))
democratic['SP500選後一年'] = democratic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 12))
democratic['SP500選後兩年'] = democratic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 24))

democratic['DXY選舉年初'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'DXY'].values[0]
)

democratic['DXY選舉年尾'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'DXY'].values[0]
)

def get_DXY_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'DXY'].values[0]

democratic['DXY選後半年'] = democratic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 6))
democratic['DXY選後一年'] = democratic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 12))
democratic['DXY選後兩年'] = democratic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 24))

democratic['US10Y選舉年初'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'US10Y'].values[0]
)

democratic['US10Y選舉年尾'] = democratic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'US10Y'].values[0]
)

def get_US10Y_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'US10Y'].values[0]

democratic['US10Y選後半年'] = democratic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 6))
democratic['US10Y選後一年'] = democratic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 12))
democratic['US10Y選後兩年'] = democratic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 24))

democratic['SP500_選舉年變化(%)'] = ((democratic['SP500選舉年尾'] - democratic['SP500選舉年初']) / democratic['SP500選舉年初'] * 100).round(0).astype(int).astype(str) + '%'
democratic['SP500_選後半年變化(%)'] = ((democratic['SP500選後半年'] - democratic['SP500']) / democratic['SP500'] * 100).round(0).astype(int).astype(str) + '%'
democratic['SP500_選後一年變化(%)'] = ((democratic['SP500選後一年'] - democratic['SP500']) / democratic['SP500'] * 100).round(0).astype(int).astype(str) + '%'
democratic['SP500_選後兩年變化(%)'] = ((democratic['SP500選後兩年'] - democratic['SP500']) / democratic['SP500'] * 100).round(0).astype(int).astype(str) + '%'

democratic['DXY_選舉年變化'] = ((democratic['DXY選舉年尾'] - democratic['DXY選舉年初']))
democratic['DXY_選後半年變化'] = ((democratic['DXY選後半年'] - democratic['DXY']))
democratic['DXY_選後一年變化'] = ((democratic['DXY選後一年'] - democratic['DXY']))
democratic['DXY_選後兩年變化'] = ((democratic['DXY選後兩年'] - democratic['DXY']))

democratic['US10Y_選舉年變化'] = (democratic['US10Y選舉年尾'] - democratic['US10Y選舉年初'])
democratic['US10Y_選後半年變化'] = (democratic['US10Y選後半年'] - democratic['US10Y'])
democratic['US10Y_選後一年變化'] = (democratic['US10Y選後一年'] - democratic['US10Y'])
democratic['US10Y_選後兩年變化'] = (democratic['US10Y選後兩年'] - democratic['US10Y'])

democratic_change = democratic[['Date', 'SP500','DXY', 'US10Y', 'SP500_選舉年變化(%)', 'SP500_選後半年變化(%)', 'SP500_選後一年變化(%)', 'SP500_選後兩年變化(%)','DXY_選舉年變化', 'DXY_選後半年變化', 'DXY_選後一年變化', 'DXY_選後兩年變化','US10Y_選舉年變化', 'US10Y_選後半年變化', 'US10Y_選後一年變化', 'US10Y_選後兩年變化']]
democratic_change

,Date,SP500,DXY,US10Y,SP500_選舉年變化(%),SP500_選後半年變化(%),SP500_選後一年變化(%),SP500_選後兩年變化(%),DXY_選舉年變化,DXY_選後半年變化,DXY_選後一年變化,DXY_選後兩年變化,US10Y_選舉年變化,US10Y_選後半年變化,US10Y_選後一年變化,US10Y_選後兩年變化
1775,1932-11-08,7.240000,NaN,NaN,-15%,15%,35%,26%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3231,1936-11-03,17.170000,NaN,NaN,28%,-4%,-33%,-23%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4694,1940-11-05,11.160000,NaN,NaN,-15%,-16%,-13%,-15%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6157,1944-11-07,12.960000,NaN,NaN,14%,15%,32%,14%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7613,1948-11-02,16.700001,NaN,NaN,-1%,-11%,-2%,18%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12002,1960-11-08,55.110001,NaN,NaN,-3%,21%,29%,6%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13458,1964-11-03,85.180000,NaN,4.150,13%,5%,8%,-5%,NaN,NaN,NaN,NaN,0.070,0.060,0.280,0.910000
17840,1976-11-02,103.099998,106.099998,7.380,19%,-4%,-12%,-7%,1.049995,-1.689995,-5.250000,-18.849998,-0.950,0.070,0.310,1.349999
23685,1992-11-03,419.920013,88.550003,6.870,4%,5%,10%,11%,8.840004,0.769997,6.459999,-1.970001,-0.010,-0.910,-1.264,1.076000
25148,1996-11-05,714.140015,87.339996,6.273,20%,16%,32%,59%,3.419998,9.820007,8.170006,6.680000,0.832,0.380,-0.341,-1.455000


Republic win

In [35]:
republic = excel_data[excel_data['Date'].isin(republican_dates)].copy()
republic['SP500選舉年初'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'SP500'].values[0]
)

republic['SP500選舉年尾'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'SP500'].values[0]
)

def get_SP500_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'SP500'].values[0]

republic['SP500選後半年'] = republic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 6))
republic['SP500選後一年'] = republic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 12))
republic['SP500選後兩年'] = republic['Date'].apply(lambda x: get_SP500_later(x, excel_data, 24))

republic['DXY選舉年初'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'DXY'].values[0]
)

republic['DXY選舉年尾'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'DXY'].values[0]
)

def get_DXY_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'DXY'].values[0]

republic['DXY選後半年'] = republic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 6))
republic['DXY選後一年'] = republic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 12))
republic['DXY選後兩年'] = republic['Date'].apply(lambda x: get_DXY_later(x, excel_data, 24))

republic['US10Y選舉年初'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=1, day=1), 'US10Y'].values[0]
)

republic['US10Y選舉年尾'] = republic['Date'].apply(
    lambda x: excel_data.loc[excel_data['Date'] == pd.Timestamp(year=x.year, month=12, day=31), 'US10Y'].values[0]
)

def get_US10Y_later(date, df, months):
    later_date = date + pd.DateOffset(months=months)
    while later_date not in df['Date'].values:
        later_date += pd.DateOffset(days=1)
    return df.loc[df['Date'] == later_date, 'US10Y'].values[0]

republic['US10Y選後半年'] = republic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 6))
republic['US10Y選後一年'] = republic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 12))
republic['US10Y選後兩年'] = republic['Date'].apply(lambda x: get_US10Y_later(x, excel_data, 24))

republic['SP500_選舉年變化(%)'] = ((republic['SP500選舉年尾'] - republic['SP500選舉年初']) / republic['SP500選舉年初'] * 100).round(0).astype(int).astype(str) + '%'
republic['SP500_選後半年變化(%)'] = ((republic['SP500選後半年'] - republic['SP500']) / republic['SP500'] * 100).round(0).astype(int).astype(str) + '%'
republic['SP500_選後一年變化(%)'] = ((republic['SP500選後一年'] - republic['SP500']) / republic['SP500'] * 100).round(0).astype(int).astype(str) + '%'
republic['SP500_選後兩年變化(%)'] = ((republic['SP500選後兩年'] - republic['SP500']) / republic['SP500'] * 100).round(0).astype(int).astype(str) + '%'

republic['DXY_選舉年變化'] = ((republic['DXY選舉年尾'] - republic['DXY選舉年初']))
republic['DXY_選後半年變化'] = ((republic['DXY選後半年'] - republic['DXY']))
republic['DXY_選後一年變化'] = ((republic['DXY選後一年'] - republic['DXY']))
republic['DXY_選後兩年變化'] = ((republic['DXY選後兩年'] - republic['DXY']))

republic['US10Y_選舉年變化'] = (republic['US10Y選舉年尾'] - republic['US10Y選舉年初'])
republic['US10Y_選後半年變化'] = (republic['US10Y選後半年'] - republic['US10Y'])
republic['US10Y_選後一年變化'] = (republic['US10Y選後一年'] - republic['US10Y'])
republic['US10Y_選後兩年變化'] = (republic['US10Y選後兩年'] - republic['US10Y'])

republic_change = republic[['Date', 'SP500','DXY', 'US10Y', 'SP500_選舉年變化(%)', 'SP500_選後半年變化(%)', 'SP500_選後一年變化(%)', 'SP500_選後兩年變化(%)','DXY_選舉年變化', 'DXY_選後半年變化', 'DXY_選後一年變化', 'DXY_選後兩年變化','US10Y_選舉年變化', 'US10Y_選後半年變化', 'US10Y_選後一年變化', 'US10Y_選後兩年變化']]
republic_change

,Date,SP500,DXY,US10Y,SP500_選舉年變化(%),SP500_選後半年變化(%),SP500_選後一年變化(%),SP500_選後兩年變化(%),DXY_選舉年變化,DXY_選後半年變化,DXY_選後一年變化,DXY_選後兩年變化,US10Y_選舉年變化,US10Y_選後半年變化,US10Y_選後一年變化,US10Y_選後兩年變化
312,1928-11-06,22.240000,NaN,NaN,38%,18%,-7%,-25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9076,1952-11-04,24.600000,NaN,NaN,12%,2%,0%,33%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10539,1956-11-06,47.599998,NaN,NaN,3%,-3%,-15%,10%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14921,1968-11-05,103.099998,NaN,5.630,8%,1%,-5%,-18%,NaN,NaN,NaN,NaN,0.460,0.560000,1.340,1.570
16384,1972-11-07,113.980003,109.809998,6.270,16%,-3%,-7%,-34%,-1.070000,-8.320000,-12.479996,-9.320000,0.520,0.540000,0.490,1.470
19303,1980-11-04,129.039993,88.639999,12.460,26%,1%,-3%,10%,4.570000,13.160004,15.980003,36.739998,2.100,2.010000,1.720,-2.040
20766,1984-11-06,170.410004,142.320007,11.600,1%,6%,13%,44%,19.680008,12.199997,-13.920013,-33.250008,-0.270,-0.410001,-1.670,-4.280
22229,1988-11-08,275.149994,93.650002,8.910,12%,11%,23%,12%,7.080002,5.309998,4.479996,-11.510002,0.310,0.130000,-1.040,-0.330
26611,2000-11-07,1431.869995,115.519997,5.859,-10%,-12%,-22%,-37%,7.689995,-0.199997,-0.539993,-10.549995,-1.325,-0.654000,-1.658,-1.977
28067,2004-11-02,1130.560059,85.470001,4.075,9%,3%,7%,21%,-6.070000,-0.980003,4.250000,-0.130005,-0.041,0.119000,0.535,0.521
